In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
kaushil268_disease_prediction_using_machine_learning_path = kagglehub.dataset_download('kaushil268/disease-prediction-using-machine-learning')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/disease-prediction-using-machine-learning/Training.csv")



In [ ]:
test = pd.read_csv("/kaggle/input/disease-prediction-using-machine-learning/Testing.csv")


In [ ]:
test.head()

In [ ]:
df.head()

In [ ]:
df = df.drop(columns=["Unnamed: 133"])

In [ ]:
df.dtypes

In [ ]:
df["prognosis"].value_counts()

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt


X = df.drop(columns=["prognosis"])
y = df["prognosis"]


In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)


In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y_encoded)



In [ ]:
importances = rf.feature_importances_
feature_names = X.columns

importance_df = pd.DataFrame({"Feature": feature_names, "Importance": importances})
importance_df = importance_df.sort_values(by="Importance", ascending=False)
print(importance_df.head(50))


In [ ]:
plt.figure(figsize=(10, 15))
plt.barh(importance_df["Feature"][:50], importance_df["Importance"][:50], color="skyblue")
plt.xlabel("Importance Score")
plt.ylabel("Features")
plt.title("Top 50 Important Features")
plt.gca().invert_yaxis()
plt.show()


In [ ]:


top_50_features = importance_df["Feature"].iloc[:50].values

X = df[top_50_features]
y = df["prognosis"]



In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [ ]:
rf = RandomForestClassifier(n_estimators=40, random_state=42)
rf.fit(X_train, y_train)

rf_predictions = rf.predict(X_test)
print("Random Forest Classification Report:")
print(classification_report(y_test, rf_predictions, target_names=le.classes_))
print(f"Random Forest Accuracy: {accuracy_score(y_test, rf_predictions):.2f}")

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, X, y_encoded, cv=5, scoring='accuracy')
print("Cross-Validation Accuracy Scores:", scores)
print("Mean Accuracy:", scores.mean())


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


test2 = test[top_50_features]

predictions_encoded = rf.predict(test2)
predictions = le.inverse_transform(predictions_encoded)

count = 0
for i, disease in enumerate(predictions):
    print(f"Row {i+1}: Predicted Disease - {disease}")
    if count == 10:
        break
    count += 1


In [ ]:
test["prognosis"].head(10)

In [ ]:
predictions = le.inverse_transform(predictions_encoded)
test['predicted_prognosis'] = predictions

mismatches = (test['prognosis'] != test['predicted_prognosis'])
mismatch_count = mismatches.sum()



In [ ]:
print(test[['prognosis', 'predicted_prognosis']].head(10))
print(f"Total mismatches: {mismatch_count} out of {len(test)} samples.")